<a href="https://colab.research.google.com/github/Moe-phantom/Moe-phantom/blob/main/kepler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
    from google.colab import files
    uploaded = files.upload()

Saving cumulative_2025.09.25_07.14.54.csv to cumulative_2025.09.25_07.14.54.csv


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import joblib

In [ ]:
df = pd.read_csv('cumulative_2025.09.25_07.14.54.csv',
                 comment='#',            # ignore lines that start with #
                 on_bad_lines='skip')    # drop any malformed rows
df.head(20)

,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,CANDIDATE,CANDIDATE,0.000,0,0,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
5,10872983,K00756.01,Kepler-228 d,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-232.0,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
6,10872983,K00756.02,Kepler-228 c,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-232.0,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
7,10872983,K00756.03,Kepler-228 b,CONFIRMED,CANDIDATE,0.992,0,0,0,0,...,-232.0,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
8,6721123,K00114.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,1,0,...,-124.0,3.986,0.182,-0.098,1.958,0.322,-0.483,298.86435,42.151569,12.660
9,10910878,K00757.01,Kepler-229 c,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-83.0,4.485,0.083,-0.028,0.848,0.033,-0.072,286.99948,48.375790,15.841


In [ ]:
!pip install lightgbm lightkurve

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/

In [ ]:
df = df[df['koi_disposition'].isin(['CONFIRMED','CANDIDATE','FALSE POSITIVE'])]

In [ ]:
label_map = {'CONFIRMED': 2, 'CANDIDATE': 1, 'FALSE POSITIVE': 0}
df['label'] = df['koi_disposition'].map(label_map)

In [ ]:
assert df['label'].isna().sum() == 0

In [ ]:
df.head()

,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,label
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,2
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,2
2,10811496,K00753.01,NaN,CANDIDATE,CANDIDATE,0.000,0,0,0,0,...,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436,1
3,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597,0
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509,2


In [ ]:
features = [
    'koi_fpflag_nt','koi_fpflag_ss','koi_fpflag_co','koi_fpflag_ec',
    'koi_period','koi_depth','koi_duration','koi_model_snr',
    'koi_prad','koi_srad','koi_impact','koi_steff','koi_slogg'
]

X = df[features]
y = df['label']

In [ ]:
X = X.fillna(X.median())
X.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_depth,koi_duration,koi_model_snr,koi_prad,koi_srad,koi_impact,koi_steff,koi_slogg
0,0,0,0,0,9.488036,615.8,2.95750,35.8,2.26,0.927,0.146,5455.0,4.467
1,0,0,0,0,54.418383,874.8,4.50700,25.8,2.83,0.927,0.586,5455.0,4.467
2,0,0,0,0,19.899140,10829.0,1.78220,76.3,14.60,0.868,0.969,5853.0,4.544
3,0,1,0,0,1.736952,8079.2,2.40641,505.6,33.46,0.791,1.276,5805.0,4.564
4,0,0,0,0,2.525592,603.3,1.65450,40.9,2.75,1.046,0.701,6031.0,4.438


In [ ]:
num_rows = df.shape[0]
print(f"The number of rows in the DataFrame is: {num_rows}")

The number of rows in the DataFrame is: 9564


In [ ]:
print(df['koi_disposition'].value_counts())

koi_disposition
FALSE POSITIVE    4839
CONFIRMED         2746
CANDIDATE         1979
Name: count, dtype: int64


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
clf = lgb.LGBMClassifier(
    n_estimators=600,
    boosting_type='dart',
    learning_rate=0.05,
    num_leaves=31,
    objective='multiclass',
    num_class=3,
    random_state=42
)

In [ ]:
clf.fit(X_train, y_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001028 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2304
[LightGBM] [Info] Number of data points in the train set: 6694, number of used features: 13
[LightGBM] [Info] Start training from score -0.681267
[LightGBM] [Info] Start training from score -1.575511
[LightGBM] [Info] Start training from score -1.247845


LGBMClassifier(boosting_type='dart', learning_rate=0.05, n_estimators=600,
               num_class=3, objective='multiclass', random_state=42)

In [ ]:
y_pred_proba = clf.predict_proba(X_test)
y_pred = clf.predict(X_test)

In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label_map.keys()))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
                 precision    recall  f1-score   support

     CONFIRMED       0.99      0.99      0.99      1452
     CANDIDATE       0.83      0.83      0.83       594
FALSE POSITIVE       0.89      0.89      0.89       824

      accuracy                           0.93      2870
     macro avg       0.90      0.90      0.90      2870
  weighted avg       0.93      0.93      0.93      2870

Confusion Matrix:
 [[1437   14    1]
 [   9  493   92]
 [   6   84  734]]


In [ ]:


# Metrics


# AUC (macro average)
auc_macro = roc_auc_score(y_test, y_pred_proba, multi_class='ovo', average='macro')
print("Macro AUC:", auc_macro)

Macro AUC: 0.975956705023075


In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="mlogloss",
    scale_pos_weight=1  # can tune for imbalance
)


In [ ]:

xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
y_proba = xgb.predict_proba(X_test)


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [08:01:05] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [ ]:

print(classification_report(y_test, y_pred))
print("Macro AUC:", roc_auc_score(y_test, y_proba, multi_class="ovr", average="macro"))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1452
           1       0.84      0.82      0.83       594
           2       0.89      0.90      0.89       824

    accuracy                           0.93      2870
   macro avg       0.91      0.90      0.90      2870
weighted avg       0.93      0.93      0.93      2870

Macro AUC: 0.9837648977760596


In [ ]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[1437   14    1]
 [  12  490   92]
 [   6   79  739]]


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Scale only X, not y
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    activation='relu',
    solver='adam',
    max_iter=300,
    random_state=42
)

mlp.fit(X_train_scaled, y_train)
y_pred = mlp.predict(X_test_scaled)
y_proba = mlp.predict_proba(X_test_scaled)

print(classification_report(y_test, y_pred))
print("Macro AUC:", roc_auc_score(y_test, y_proba, multi_class="ovr", average="macro"))


              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1452
           1       0.80      0.85      0.82       594
           2       0.89      0.86      0.87       824

    accuracy                           0.92      2870
   macro avg       0.89      0.90      0.89      2870
weighted avg       0.92      0.92      0.92      2870

Macro AUC: 0.9772160028199249


In [ ]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[1426   18    8]
 [   8  502   84]
 [   9  107  708]]


In [ ]:
df = pd.read_csv('/content/TOI_2025.09.28_05.51.22.csv',
                 comment='#',            # ignore lines that start with #
                 on_bad_lines='skip')    # drop any malformed rows
df.head(20)

,rowid,toi,toipfx,tid,ctoi_alias,pl_pnum,tfopwg_disp,rastr,ra,decstr,...,pl_rade,pl_insol,pl_eqt,st_tmag,st_dist,st_teff,st_logg,st_rad,toi_created,rowupdate
0,1,1000.01,1000,50365310,5.036531e+07,1,FP,07h29m25.85s,112.357708,-12d41m45.46s,...,5.818163,22601.948581,3127.204052,9.604000,485.735,10249.0,4.19000,2.169860,2019-07-24 15:58:33,2024-09-09 10:08:01
1,2,1001.01,1001,88863718,8.886372e+07,1,PC,08h10m19.31s,122.580465,-05d30m49.87s,...,11.215400,44464.500000,4045.000000,9.423440,295.862,7070.0,4.03000,2.010000,2019-07-24 15:58:33,2023-04-03 14:31:04
2,3,1002.01,1002,124709665,1.247097e+08,1,FP,06h58m54.47s,104.726966,-10d34m49.64s,...,23.752900,2860.610000,2037.000000,9.299501,943.109,8924.0,NaN,5.730000,2019-07-24 15:58:33,2022-07-11 16:02:02
3,4,1003.01,1003,106997505,1.069975e+08,1,FP,07h22m14.39s,110.559945,-25d12m25.26s,...,NaN,1177.360000,1631.000000,9.300300,7728.170,5388.5,4.15000,NaN,2019-07-24 15:58:33,2022-02-23 10:10:02
4,5,1004.01,1004,238597883,2.385979e+08,1,FP,08h08m42.77s,122.178195,-48d48m10.12s,...,11.311300,54679.300000,4260.000000,9.135500,356.437,9219.0,4.14000,2.150000,2019-07-24 15:58:33,2024-09-09 10:08:01
5,6,1005.01,1005,169904935,1.699049e+08,1,FP,08h02m49.15s,120.704811,-11d06m05.48s,...,7.108410,414.775000,1257.000000,9.130900,100.711,5613.0,NaN,1.090000,2019-07-24 15:58:33,2025-07-08 10:08:01
6,7,1006.01,1006,156115721,1.561157e+08,1,FP,08h17m26.22s,124.359239,-27d16m24.68s,...,8.695090,1750.932635,1649.822070,9.238600,NaN,6616.0,NaN,1.534290,2019-07-24 15:58:33,2021-10-29 12:59:15
7,8,1007.01,1007,65212867,6.521287e+07,1,PC,07h31m00.57s,112.752393,-04d27m48.09s,...,14.775200,448.744000,1282.000000,8.877590,283.291,6596.0,3.71000,2.700000,2019-07-24 15:58:33,2021-10-29 12:59:15
8,9,1008.01,1008,440801822,4.408018e+08,1,FP,07h17m31.88s,109.382839,+13d23m42.79s,...,3.196210,5644.400000,NaN,8.936200,144.297,6858.5,4.20000,1.570000,2019-07-24 15:58:33,2023-12-08 16:02:02
9,10,1009.01,1009,107782586,1.077826e+08,1,PC,07h26m40.28s,111.667845,-24d27m43.6s,...,NaN,1063.342296,1456.424653,8.433100,866.147,8868.7,NaN,NaN,2019-07-24 15:58:33,2024-01-17 10:08:02


In [ ]:
tess_column_names = df.columns.tolist()
print(tess_column_names)

['rowid', 'toi', 'toipfx', 'tid', 'ctoi_alias', 'pl_pnum', 'tfopwg_disp', 'rastr', 'ra', 'decstr', 'dec', 'st_pmra', 'st_pmdec', 'pl_tranmid', 'pl_orbper', 'pl_trandurh', 'pl_trandep', 'pl_rade', 'pl_insol', 'pl_eqt', 'st_tmag', 'st_dist', 'st_teff', 'st_logg', 'st_rad', 'toi_created', 'rowupdate']


In [ ]:
unique_disp_values = df['tfopwg_disp'].unique()
print(unique_disp_values)

['FP' 'PC' 'KP' 'APC' 'FA' 'CP']
